### ISyE524: Intro to Optimization
#### Problem Set #5
##### 1. Ahhhhhhhh, Daniel-son. You SuDoku Master
###### 1-1

\begin{align*}
\text{decision variables: } \\
\quad & X_{ijk} = 
  \begin{cases} 
   1 & \text{if } (i,j) \text{ entry is a } k \\
   0 & \text{otherwise} 
  \end{cases} \\

\text{constraints: } \\

\quad & \text{1. cell constraints:} \\
& \sum_{k=1}^{n} x_{ijk} = 1 \quad \forall i, j \in {1, ..., n}\\
& \text{2. column constraints:} \\
& \sum_{i=1}^{n} x_{ijk} = 1 \quad \forall j, k \in {1, ..., n}\\
& \text{3. row constraints:} \\
& \sum_{j=1}^{n} x_{ijk} = 1 \quad \forall i, k \in {1, ..., n}\\
& \text{4. cluster constraints:} \\
& \sum_{i=p}^{p+\sqrt{n}-1}\sum_{j=q}^{q+\sqrt{n}-1} x_{ijk} = 1 \quad \forall p, q \in{1, \sqrt{n}+1, 2\sqrt{n}+1, ..., n - \sqrt{n}+1}, k \in {1, ..., n}\\
& \text{5. fixed position constraints:} \\
& x_{ijk} = 1 \quad \forall F_k \\
\end{align*}

###### 1-2

In [1]:
# Given data. Unknown entries are specified as "0"
given = [
    0 6 0  1 0 4  0 5 0
    0 0 8  3 0 5  6 0 0
    2 0 0  0 0 0  0 0 1
 
    8 0 0  4 0 7  0 0 6
    0 0 6  0 0 0  3 0 0
    7 0 0  9 0 1  0 0 4
 
    5 0 0  0 0 0  0 0 2
    0 0 7  2 0 6  9 0 0
    0 4 0  5 0 8  0 7 0
  ];
# helper function to print a sudoku grid
function printSudoku(arr)
    u = 0
    println("+-------+-------+-------+")
    for p in 1:3:9
      for q in 0:2
        print("| ")
        for r in 1:3:9
          for s in 0:2
            u = round(Int, arr[p+q,r+s])
            u == 0 ? print(" ") : print(u)
            print(" ")
          end
          print("| ")
                  end
        println()
      end
      println("+-------+-------+-------+")
    end
  end
  ;

In [2]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, x[1:9,1:9,1:9], Bin)

# exactly one number per cell
for i in 1:9
  for j in 1:9
    @constraint(m, sum(x[i,j,k] for k in 1:9) == 1)
  end
end

# exactly one of each number per row
for i in 1:9
  for k in 1:9
    @constraint(m, sum(x[i,j,k] for j in 1:9) == 1)
  end
end

# exactly one of each number per column
for j in 1:9
  for k in 1:9
    @constraint(m, sum(x[i,j,k] for i in 1:9) == 1)
  end
end

# exactly one of each number per 3x3 block
for k in 1:9
  for p in 0:3:6
    for q in 0:3:6
      @constraint(m, sum(x[p+i,q+j,k] for i in 1:3, j in 1:3) == 1)
    end
  end
end

# initial conditions
for i in 1:9
  for j in 1:9
    if given[i,j] != 0
      @constraint(m, x[i,j,given[i,j]] == 1)
    end
  end
end

@time(optimize!(m))

# if termination_status(m) != :OPTIMAL
#     println(termination_status(m))
# else
    #generate solution grid and display the solution
    solution = zeros(9,9)
    for i in 1:9
      for j in 1:9
        for k in 1:9
          if value(x[i,j,k]) == 1
            solution[i,j] = k
            continue
          end
        end
      end
    end

    println("The solution is: ")
    printSudoku(solution)
# end

  0.100060 seconds (180.07 k allocations: 12.679 MiB, 82.88% compilation time: 94% of which was recompilation)
The solution is: 
+-------+-------+-------+
| 9 6 3 | 1 7 4 | 2 5 8 | 
| 1 7 8 | 3 2 5 | 6 4 9 | 
| 2 5 4 | 6 8 9 | 7 3 1 | 
+-------+-------+-------+
| 8 2 1 | 4 3 7 | 5 9 6 | 
| 4 9 6 | 8 5 2 | 3 1 7 | 
| 7 3 5 | 9 6 1 | 8 2 4 | 
+-------+-------+-------+
| 5 8 9 | 7 1 3 | 4 6 2 | 
| 3 1 7 | 2 4 6 | 9 8 5 | 
| 6 4 2 | 5 9 8 | 1 7 3 | 
+-------+-------+-------+


###### 1-3

\begin{align*}
\text{objective function: } \\
\quad & \text{maximize: } \sum_{i=1}^{n}\sum_{k=1}^{n} x_{iik} \times k \\

\text{decision variables: } \\
\quad & X_{ijk} = 
  \begin{cases} 
   1 & \text{if } (i,j) \text{ entry is a } k \\
   0 & \text{otherwise} 
  \end{cases} \\

\text{constraints: } \\

\quad & \text{1. cell constraints:} \\
& \sum_{k=1}^{n} x_{ijk} = 1 \quad \forall i, j \in {1, ..., n}\\
& \text{2. column constraints:} \\
& \sum_{i=1}^{n} x_{ijk} = 1 \quad \forall j, k \in {1, ..., n}\\
& \text{3. row constraints:} \\
& \sum_{j=1}^{n} x_{ijk} = 1 \quad \forall i, k \in {1, ..., n}\\
& \text{4. cluster constraints:} \\
& \sum_{i=p}^{p+\sqrt{n}-1}\sum_{j=q}^{q+\sqrt{n}-1} x_{ijk} = 1 \quad \forall p, q \in{1, \sqrt{n}+1, 2\sqrt{n}+1, ..., n - \sqrt{n}+1}, k \in {1, ..., n}\\
& \text{5. fixed position constraints:} \\
& \sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n} x_{ijk} \geq K \quad \forall F_k \\
\end{align*}

###### 1-4

In [5]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, x[1:9,1:9,1:9], Bin)
@objective(m, Max, sum(k*x[i,i,k] for i in 1:9, k in 1:9))

# exactly one number per cell
for i in 1:9
  for j in 1:9
    @constraint(m, sum(x[i,j,k] for k in 1:9) == 1)
  end
end

# exactly one of each number per row
for i in 1:9
  for k in 1:9
    @constraint(m, sum(x[i,j,k] for j in 1:9) == 1)
  end
end

# exactly one of each number per column
for j in 1:9
  for k in 1:9
    @constraint(m, sum(x[i,j,k] for i in 1:9) == 1)
  end
end

# exactly one of each number per 3x3 block
for k in 1:9
  for p in 0:3:6
    for q in 0:3:6
      @constraint(m, sum(x[p+i,q+j,k] for i in 1:3, j in 1:3) == 1)
    end
  end
end

# initial conditions
# for i in 1:9
#   for j in 1:9
#     if given[i,j] != 0
#       @constraint(m, x[i,j,given[i,j]] == 1)
#     end
#   end
# end

# @constraint(m, sum(x[i,j,k] for i in 1:9, j in 1:9, k in 1:9) >= 24)

# Create a binary decision variable matrix of the same size as `given`
@variable(m, supp[1:size(given, 1), 1:size(given, 2)], Bin)
# Create a constraint that requires exactly 24 of the non-zero elements in `given` to match the final solution
@constraint(m, sum(supp[i, j] for i in 1:size(given, 1), j in 1:size(given, 2) if given[i, j] != 0) == 24)


@time(optimize!(m))

# if termination_status(m) != :OPTIMAL
#     println(termination_status(m))
# else
    #generate solution grid and display the solution
    solution = zeros(9,9)
    for i in 1:9
      for j in 1:9
        for k in 1:9
          if value(x[i,j,k]) == 1
            solution[i,j] = k
            continue
          end
        end
      end
    end

    println("The solution is: ")
    printSudoku(solution)
# end

  6.175432 seconds (6.38 k allocations: 1.247 MiB)
The solution is: 
+-------+-------+-------+
| 9 5 3 | 4 8 7 | 1 6 2 | 
| 2 8 1 | 3 9 6 | 5 4 7 | 
| 4 6 7 | 5 2 1 | 3 8 9 | 
+-------+-------+-------+
| 3 2 5 | 9 1 4 | 8 7 6 | 
| 8 4 9 | 6 7 3 | 2 1 5 | 
| 7 1 6 | 2 5 8 | 9 3 4 | 
+-------+-------+-------+
| 6 3 2 | 8 4 9 | 7 5 1 | 
| 5 7 8 | 1 6 2 | 4 9 3 | 
| 1 9 4 | 7 3 5 | 6 2 8 | 
+-------+-------+-------+


###### 1-5

\begin{align*}
& \text{new if-then constraints: } \\

& \text{if put 2 or more “9”s on the main diagonal, then y=1: } \\
& \sum_{i=1}^{n} x_{ii9} - 2 \leq (n - 2) \times y -  (1-y) \\

& \text{which is: } \\
& \sum_{i=1}^{n} x_{ii9} \leq 3 + (n - 1) \times y \\

& \text{if y=1, then must also put exactly 3 “5”’s on the diagonal: } \\
& \sum_{i=1}^{n} x_{ii5} - 3 = (-3) \times (1-y) \\

& \text{which is: } \\
& \sum_{i=1}^{n} x_{ii5} = 3y \\



\end{align*}

##### 2. Integer Programming is Smarter than a Fourth Grader?
###### 2-1

\begin{align*}
\text{decision variables: } \\
\quad & xD{g,d} = 
  \begin{cases} 
   1 & \text{if Grandma g has Daughter d } \\
   0 & \text{otherwise} 
  \end{cases} \\

\quad & xH{g,h} = 
  \begin{cases} 
   1 & \text{if Grandma g has Son-In-Law h } \\
   0 & \text{otherwise} 
  \end{cases} \\

\quad & xS{g,s} = 
  \begin{cases} 
   1 & \text{if Grandma g has Grandson s } \\
   0 & \text{otherwise} 
  \end{cases} \\



\text{constraints: } \\

\quad & \text{1. G-D constraints:} \\
& \sum_{g \in G} xD_{gd} = 1 \quad \forall d \in D\\
& \sum_{d \in D} xD_{gd} = 1 \quad \forall g \in G\\
& \text{2. G-H constraints:} \\
& \sum_{g \in G} xH_{gh} = 1 \quad \forall h \in H\\
& \sum_{h \in H} xH_{gh} = 1 \quad \forall g \in G\\
& \text{3. G-S constraints:} \\
& \sum_{g \in G} xS_{gs} = 1 \quad \forall s \in S\\
& \sum_{s \in S} xS_{gs} = 1 \quad \forall g \in G\\

& \text{4. Maxine’s Daughter was not Carla} \\
& xD_{Maxine, Carla} = 0 \\

& \text{5. Mavis’ son-in-law was not named Jack, and Cathy was married to Joe, but their son was not named Tab} \\
& xH_{Mavis, Jake} = 0 \\
& xH_{Cathy's mother, Joe} = 1 \\
& xS_{Cathy's mother, Tab} = 0 \\

& \text{6. Tim’s mother was not named either Carol or Carla, because Tim’s mother was Jake’s wife Cindy} \\
& xS_{Carol' mother, Tim} = 0 \\
& xS_{Carla' mother, Tim} = 0 \\
& xS_{Cindy's mother, Tim} = 1 \\
& xH_{Cindy's mother, Jake} = 1 \\

& \text{7. Mabel, Millie, and Martha did not have daughters named Carla or Carol, and Martha’s son-in-law
was not John because John was married to Caren, and their son was named Tom} \\
& xD_{Mabel, Carla} = 0 \\
& xD_{Mabel, Carol} = 0 \\
& xD_{Millie, Carla} = 0 \\
& xD_{Millie, Carol} = 0 \\
& xD_{Martha, Carla} = 0 \\
& xD_{Martha, Carol} = 0 \\
& xH_{Martha, John} = 0 \\
& xS_{Caren's mother, Tom} = 1 \\
& xH_{Caren's mother, John} = 1 \\

& \text{8. Millie was terrible with names, but she knew that her son-in-law was named either Joe or Jason,
and her grandson was named either Tip or Tab} \\
& xH_{Millie, Joe} + xH_{Millie, Jason} = 1 \\
& xS_{Millie, Tip} + xS_{Millie, Tab} = 1 \\

& \text{9. Tab did not have a grandmother named Mavis} \\
& xS_{Mavis, Tab} = 0 \\
\end{align*}

###### 2-2

In [22]:
G = [:Maxine, :Mabel, :Mavis, :Millie, :Martha]
D = [:Carla, :Carol, :Cindy, :Cathy, :Caren ]
H = [:John, :Jake, :Jack, :Joe, :Jason ]
S = [:Tom, :Tex, :Tim, :Tip, :Tab ]

function printGrandmaSolution(xD, xH, xS)
    for g in G
        grandma = g
        daughter = :Unknown
        son_in_law = :Unknown
        grandson = :Unknown 

        for d in D 
            if value(xD[g,d]) > 0.5
                daughter = d 
            end
        end
        for h in H 
            if value(xH[g,h]) > 0.5
                son_in_law = h 
            end
        end
        for s in S 
            if value(xS[g,s]) > 0.5
                grandson = s 
            end
        end
        println("Grandma ", g, " has daughter ", daughter, " son-in-law ", son_in_law, " and grandson ", grandson)
    end
end; 

In [61]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
set_silent(m)

@variable(m, xD[G,D], Bin)
@variable(m, xH[G,H], Bin)
@variable(m, xS[G,S], Bin)

# 1. GD
for d in D
    @constraint(m, sum(xD[g, d] for g in G) == 1)
end

for g in G
    @constraint(m, sum(xD[g, d] for d in D) == 1)
end

# 2. GH
for h in H
    @constraint(m, sum(xH[g, h] for g in G) == 1)
end

for g in G
    @constraint(m, sum(xH[g, h] for h in H) == 1)
end


# 3. GS
for s in S
    @constraint(m, sum(xS[g, s] for g in G) == 1)
end

for g in G
    @constraint(m, sum(xS[g, s] for s in S) == 1)
end

# 4. xD_{Maxine, Carla} = 0 
@constraint(m, xD[:Maxine, :Carla] == 0)

# 5. & xH_{Mavis, Jake} = 0 \\
@constraint(m, xH[:Mavis, :Jake] == 0)
#    & xH_{Cathy's mother, Joe} = 1 \\
for g in G
    @constraint(m, xH[g, :Joe] == xD[g, :Cathy])
end
#    & xS_{Cathy's mother, Tab} = 0 \\
for g in G
    @constraint(m, xS[g, :Tab] + xD[g, :Cathy] <= 1)
end

# 6. & xS_{Carol' mother, Tim} = 0 \\
for g in G
    @constraint(m, xS[g, :Tim] + xD[g, :Carol] <= 1)
end
#   & xS_{Carla' mother, Tim} = 0 \\
for g in G
    @constraint(m, xS[g, :Tim] + xD[g, :Carla] <= 1)
end
#   & xS_{Cindy's mother, Tim} = 1 \\
for g in G
    @constraint(m, xS[g, :Tim] == xD[g, :Cindy])
end
#   & xH_{Cindy's mother, Jake} = 1 \\
for g in G
    @constraint(m, xH[g, :Jake] == xD[g, :Cindy])
end

# 7. & xD_{Mabel, Carla} = 0 \\
@constraint(m, xD[:Mabel, :Carla] == 0)
#   & xD_{Mabel, Carol} = 0 \\
@constraint(m, xD[:Mabel, :Carol] == 0)
#   & xD_{Millie, Carla} = 0 \\
@constraint(m, xD[:Millie, :Carla] == 0)
#   & xD_{Millie, Carol} = 0 \\
@constraint(m, xD[:Millie, :Carol] == 0)
#   & xD_{Martha, Carla} = 0 \\
@constraint(m, xD[:Martha, :Carla] == 0)
#   & xD_{Martha, Carol} = 0 \\
@constraint(m, xD[:Martha, :Carol] == 0)
#   & xH_{Martha, John} = 0 \\
@constraint(m, xH[:Martha, :John] == 0)
#   & xS_{Caren's mother, Tom} = 1 \\
for g in G
    @constraint(m, xS[g, :Tom] == xD[g, :Caren])
end
#   & xH_{Caren's mother, John} = 1 \\
for g in G
    @constraint(m, xH[g, :John] == xD[g, :Caren])
end

# 8. & xH_{Millie, Joe} + xH_{Millie, Jason} = 1 \\
@constraint(m, xH[:Millie, :Joe] + xH[:Millie, :Jason] == 1)
#   & xS_{Millie, Tip} + xS_{Millie, Tab} = 1 \\
@constraint(m, xS[:Millie, :Tip] + xS[:Millie, :Tab] == 1)

# 9. xS_{Mavis, Tab} = 0 \\
@constraint(m, xS[:Mavis, :Tab] == 0)

# Solve the model
optimize!(m)

if termination_status(m) == MOI.OPTIMAL
    println("A solution has been found!")
else
    println("The puzzle does not have a solution.")
end

printGrandmaSolution(xD, xH, xS)

A solution has been found!
Grandma Maxine has daughter Carol son-in-law Jason and grandson Tab
Grandma Mabel has daughter Caren son-in-law John and grandson Tom
Grandma Mavis has daughter Carla son-in-law Jack and grandson Tex
Grandma Millie has daughter Cathy son-in-law Joe and grandson Tip
Grandma Martha has daughter Cindy son-in-law Jake and grandson Tim


##### 3. An Auror is Near

###### 3-1 （set covering trail）
set of districts: D={1,2,3,4,5,6,7,8}

x_i=1 if district i is coverd.

z_i=1 if an auror is in district i.

\begin{align*}
\text{objective function: } \\
\quad & \text{maximize: } \sum_{i=1}^{8} p_{i}x_{i} \\

\text{constraints: } \\

\quad & \sum_{i=1}^{8} z_{i} = 3 \\
\quad & x_{i} \leq \sum_{j\in D, T{ij} \leq 2} z_{j} \quad \forall i \in D\\

& z_{i}, x_{i} \in {0, 1} \quad \forall i \in D \\

\end{align*}

###### 3-2

In [6]:
using JuMP, GLPK

# Given data
p = [40, 30, 35, 20, 15, 50, 45, 60]
T = [
  0 3 4 6 1 9 8 10;
  3 0 5 4 8 6 1 9;
  4 5 0 2 2 3 5 7;
  6 4 2 0 3 2 5 4;
  1 8 2 3 0 2 2 4;
  9 6 3 2 2 0 3 2;
  8 1 5 5 2 3 0 2;
  10 9 7 4 4 2 2 0;
]

# Model
model = Model(GLPK.Optimizer)

# Decision variables
@variable(model, x[1:8], Bin)
@variable(model, z[1:8], Bin)

# Objective
@objective(model, Max, sum(p[i] * x[i] for i in 1:8))

# Constraints
# Only three auror stations can be opened
@constraint(model, sum(z[i] for i in 1:8) == 3)

# Coverage constraint
for i in 1:8
    @constraint(model, x[i] <= sum(z[j] for j in 1:8 if T[i,j] <= 2))
end

# Solve the problem
optimize!(model)

# Output the solution
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found:")
    for i in 1:8
        if value(z[i]) > 0.5
            println("An auror is located in district $i")
        end
    end
    for i in 1:8
        if value(x[i]) > 0.5
            println("District $i is covered")
        end
    end
else
    println("No optimal solution found.")
end


Optimal solution found:
An auror is located in district 2
An auror is located in district 5
An auror is located in district 6
District 1 is covered
District 2 is covered
District 3 is covered
District 4 is covered
District 5 is covered
District 6 is covered
District 7 is covered
District 8 is covered


##### 4. Routing Apple Deliveries
###### 4-1


###### 4-2

##### 5. Paint Production
###### 5-1

###### 5-2